In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine

In [3]:
labels = pd.read_csv('../data/round-3/mikey-devon-labels-round-3_validation.csv')

In [3]:
labels

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description
0,Devon Snyder,7958,1064,Addison,SurfaceProblem,41.976437,-87.979591,Vnk-r0opTiyjIyISxOMRUQ,2023-06-26 14:46:47.411-07,NaN,1,f,cracks,NaN
1,Devon Snyder,7961,1064,Addison,SurfaceProblem,41.976780,-87.979561,w2tdzPaq9Rl7drhiXjPrvw,2023-06-26 14:48:58.144-07,NaN,1,f,cracks,NaN
2,Devon Snyder,7965,1064,Addison,SurfaceProblem,41.977058,-87.979546,tfCdM8I4Mq5HZTZyleqk_g,2023-06-26 14:51:24.7-07,NaN,1,f,height difference,NaN
3,Devon Snyder,7966,1064,Addison,SurfaceProblem,41.977795,-87.979553,dq_qwSfjMcwFLphaV8qzRw,2023-06-26 14:53:31.403-07,NaN,1,f,cracks,NaN
4,Devon Snyder,7967,1064,Addison,SurfaceProblem,41.978218,-87.979507,MMoUtn6D9XovUCMh9vfZgQ,2023-06-26 14:54:10.073-07,NaN,2,f,"cracks,grass",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,mikey,9274,757,Windermere,SurfaceProblem,47.666374,-122.271454,K3ULPp5OK1YxsW4OqDsREA,2023-09-26 14:39:17.737-07,NaN,1,f,cracks,NaN
1189,mikey,9275,757,Windermere,SurfaceProblem,47.666420,-122.271591,V9gE_6NYyIJS5KcZP5eR9Q,2023-09-26 14:39:34.917-07,NaN,1,f,"utility panel,cracks",NaN
1190,mikey,9276,757,Windermere,SurfaceProblem,47.666634,-122.271835,AFkD-p2TS8W2rXBe9eyJxQ,2023-09-26 14:40:12.08-07,NaN,1,f,cracks,NaN
1191,mikey,9277,757,Windermere,SurfaceProblem,47.666695,-122.271919,OVzfrTahs2hC9i8xuElveA,2023-09-26 14:40:21.462-07,NaN,1,f,utility panel,NaN


In [4]:
#create labels geo dataframe
points = labels.apply(lambda row: Point(row.lng, row.lat), axis=1)
labels_geo = gpd.GeoDataFrame(labels, geometry=points)
labels_geo.crs = {'init': 'epsg:4326'}
labels_geo

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry
0,Devon Snyder,7958,1064,Addison,SurfaceProblem,41.976437,-87.979591,Vnk-r0opTiyjIyISxOMRUQ,2023-06-26 14:46:47.411-07,NaN,1,f,cracks,NaN,POINT (-87.97959 41.97644)
1,Devon Snyder,7961,1064,Addison,SurfaceProblem,41.976780,-87.979561,w2tdzPaq9Rl7drhiXjPrvw,2023-06-26 14:48:58.144-07,NaN,1,f,cracks,NaN,POINT (-87.97956 41.97678)
2,Devon Snyder,7965,1064,Addison,SurfaceProblem,41.977058,-87.979546,tfCdM8I4Mq5HZTZyleqk_g,2023-06-26 14:51:24.7-07,NaN,1,f,height difference,NaN,POINT (-87.97955 41.97706)
3,Devon Snyder,7966,1064,Addison,SurfaceProblem,41.977795,-87.979553,dq_qwSfjMcwFLphaV8qzRw,2023-06-26 14:53:31.403-07,NaN,1,f,cracks,NaN,POINT (-87.97955 41.97779)
4,Devon Snyder,7967,1064,Addison,SurfaceProblem,41.978218,-87.979507,MMoUtn6D9XovUCMh9vfZgQ,2023-06-26 14:54:10.073-07,NaN,2,f,"cracks,grass",NaN,POINT (-87.97951 41.97822)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,mikey,9274,757,Windermere,SurfaceProblem,47.666374,-122.271454,K3ULPp5OK1YxsW4OqDsREA,2023-09-26 14:39:17.737-07,NaN,1,f,cracks,NaN,POINT (-122.27145 47.66637)
1189,mikey,9275,757,Windermere,SurfaceProblem,47.666420,-122.271591,V9gE_6NYyIJS5KcZP5eR9Q,2023-09-26 14:39:34.917-07,NaN,1,f,"utility panel,cracks",NaN,POINT (-122.27159 47.66642)
1190,mikey,9276,757,Windermere,SurfaceProblem,47.666634,-122.271835,AFkD-p2TS8W2rXBe9eyJxQ,2023-09-26 14:40:12.08-07,NaN,1,f,cracks,NaN,POINT (-122.27184 47.66663)
1191,mikey,9277,757,Windermere,SurfaceProblem,47.666695,-122.271919,OVzfrTahs2hC9i8xuElveA,2023-09-26 14:40:21.462-07,NaN,1,f,utility panel,NaN,POINT (-122.27192 47.66669)


In [5]:
# Seperate Seattle and Chicago data
chicago = labels_geo.loc[labels_geo['lng'] > -105]
seattle = labels_geo.loc[labels_geo['lng'] < -105]

In [7]:
#export to json
chicago.to_file("../data/round-3/chicago-validation-labels-round3-230927.geojson", driver='GeoJSON')
seattle.to_file("../data/round-3/seattle-validation-labels-round3-230927.geojson", driver='GeoJSON')

In [9]:
# read in datasets/streets_all_230515.json
streets_validated = gpd.read_file("../data/round-3/mikey-devon-streets-round-3_validation.csv")
streets_validated

,route_id,region_id,street_edge_id,way_type,geom,geometry
0,27,27,1064,secondary,0102000020E610000006000000877945A6B2FE55C0B908...,None
1,27,27,1130,secondary,0102000020E6100000040000006DFDF49FB5FE55C07491...,None
2,28,25,1043,tertiary,0102000020E610000003000000BB0F9B23060456C02ABE...,None
3,28,25,1305,tertiary,0102000020E6100000060000008117C792030456C0CC10...,None
4,28,25,1030,tertiary,0102000020E61000000900000016FFD2FDF70356C03348...,None
...,...,...,...,...,...,...
104,44,2,1309,residential,0102000020E610000005000000EFD3A70B67945EC05393...,None
105,44,2,1311,secondary,0102000020E6100000060000008C03F92356945EC0240F...,None
106,44,2,1310,secondary,0102000020E610000005000000A4B8F47956945EC02A19...,None
107,44,2,1308,tertiary,0102000020E6100000070000000F5542C156945EC03815...,None


In [10]:
# drop geometry column
streets_validated = streets_validated.drop(columns=['geometry'])

In [11]:
from shapely import wkb

In [12]:
# apply wkb.loads(geom, hex=True) to the geom column
streets_validated['geom'] = streets_validated['geom'].apply(lambda x: wkb.loads(x, hex=True))
streets_validated

,route_id,region_id,street_edge_id,way_type,geom
0,27,27,1064,secondary,"LINESTRING (-87.9796539 41.9761782, -87.979703..."
1,27,27,1130,secondary,"LINESTRING (-87.9798355 41.9806404, -87.979848..."
2,28,25,1043,tertiary,"LINESTRING (-88.0628747 41.9058988, -88.062914..."
3,28,25,1305,tertiary,"LINESTRING (-88.0627181 41.9075761, -88.062722..."
4,28,25,1030,tertiary,"LINESTRING (-88.0620112 41.9209863, -88.061972..."
...,...,...,...,...,...
104,44,2,1309,residential,"LINESTRING (-122.3187894 47.659456, -122.31871..."
105,44,2,1311,secondary,"LINESTRING (-122.3177576 47.659457, -122.31776..."
106,44,2,1310,secondary,"LINESTRING (-122.3177781 47.6583844, -122.3177..."
107,44,2,1308,tertiary,"LINESTRING (-122.3177951 47.6574986, -122.3178..."


In [16]:
# create streets geo dataframe
streets_validated = gpd.GeoDataFrame(streets_validated, geometry='geom')
streets_validated.crs = {'init': 'epsg:4326'}
#streets_validated.explore()

In [14]:
#seperate seattle and chicago based on geometry, chicago >-105 and < -80, seattle < -105
streets_chicago = streets_validated.loc[streets_validated['geom'].bounds['minx'] > -105]
streets_chicago = streets_chicago.loc[streets_chicago['geom'].bounds['minx'] < -80]
streets_seattle = streets_validated.loc[streets_validated['geom'].bounds['minx'] < -105]

In [15]:
#save to geojson
streets_chicago.to_file("../data/round-3/chicago-streets-230927.geojson", driver='GeoJSON')
streets_seattle.to_file("../data/round-3/seattle-streets-230927.geojson", driver='GeoJSON')